In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#Imports
import numpy as np
import tensorflow as tf
#from tensorflow import layers
#from tensorflow.python.layers.convolutional import conv2d 
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
import glob
from scipy import misc

In [6]:
def cnn_model_fn(features,labels,mode):
    #input layer
    input_layer=tf.reshape(features,[-1,128,128,3])
    
    #Convolutional layer 1 
    conv1=tf.layers.conv2d(
    inputs=input_layer,
    filters=32,
    kernel_size=[5,5],
    padding="same",
    activation=tf.nn.relu)
    
    #pooling layer - change pool size later
    pool1=tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
    #Convolutional layer 2
    conv2=tf.layers.conv2d(
    inputs=pool1,
    filters=16,
    kernel_size=[5,5],
    padding="same",
    activation=tf.nn.relu)
    
    #pooling layer - change pool size later
    pool2=tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    
    #Convolutional layer 3
    conv3=tf.layers.conv2d(
    inputs=input_layer,
    filters=4,
    kernel_size=[5,5],
    padding="same",
    activation=tf.nn.relu)
    
    #pooling layer - change pool size later
    pool3=tf.layers.max_pooling2d(inputs=conv3,pool_size=[2,2],strides=2)
    
    #Dense layer
    # Dense Laye
    pool3_flat = tf.reshape(pool3, [-1, 64 * 64 * 4])
    dense1 = tf.layers.dense(inputs=pool3_flat, units=256, activation=tf.nn.relu)
    dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
    dense3 = tf.layers.dense(inputs=dense2, units=64, activation=tf.nn.relu)
    dense4 = tf.layers.dense(inputs=dense3, units=32, activation=tf.nn.relu)
    dense5 = tf.layers.dense(inputs=dense4, units=16, activation=tf.nn.relu)
    dense6 = tf.layers.dense(inputs=dense5, units=3, activation=tf.nn.relu)

    dropout = tf.layers.dropout(
      inputs=dense6, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

      # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=3)

    loss = None
    train_op = None

      # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=3)
        loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.01,
        optimizer="SGD")

    # Generate Predictions
    predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
  }
    

    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
          mode=mode, predictions=predictions, loss=loss, train_op=train_op)
    

    
    


In [ ]:
def main(unused_argv):
    images_train=[]
    output_train=[]
    directory='/home/ubuntu/Data2/'
    for image_path in glob.glob(directory+"*.png"):
        currentimage = misc.imread(image_path)
        currentresize = misc.imresize(currentimage,(128,128))
        if 'Building' in image_path:
            images_train.append(currentresize)
            output_train.append(np.array(0))
        elif 'Road' in image_path:
            images_train.append(currentresize)
            output_train.append(np.array(1))
        elif 'Green' in image_path:
            images_train.append(currentresize)
            output_train.append(np.array(2))
        
    images_np_array_train = np.float32(np.array(images_train))
    output_np_array_train = np.float32(np.array(output_train))

    
    images_test=[]
    output_test=[]
    directory='/home/ubuntu/Test2/'
    for image_path in glob.glob(directory+"*.png"):
        currentimage = misc.imread(image_path)
        currentresize = misc.imresize(currentimage,(128,128))
        if 'Building' in image_path:
            images_test.append(currentresize)
            output_test.append(np.array(0))
        elif 'Road' in image_path:
            images_test.append(currentresize)
            output_test.append(np.array(1))
        elif 'Green' in image_path:
            images_test.append(currentresize)
            output_test.append(np.array(2))
    #plt.imshow(images_test[0])
    images_np_array_test = np.float32(np.array(images_test))
    output_np_array_test = np.float32(np.array(output_test))

   
    #creating a estimator
    classifier=learn.Estimator(model_fn=cnn_model_fn)
    #logging
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    classifier.fit(x=images_np_array_train,y=output_np_array_train,batch_size=25,steps=20000,monitors=[logging_hook])
    #Accuracy
    metrics = {"accuracy":learn.MetricSpec(metric_fn=tf.metrics.accuracy, prediction_key="classes"),}
    eval_results=classifier.evaluate(
    x=images_np_array_test,y=output_np_array_test,metrics=metrics)
    print(eval_results)
    
if __name__=="__main__":
    tf.app.run()




INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f48226a00f0>, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_environment': 'local', '_num_ps_replicas': 0, '_task_id': 0, '_is_chief': True, '_task_type': None, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_master': ''}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y 

/usr/local/lib/python3.4/dist-packages/tensorflow/python/util/deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmptusz5418/model.ckpt.
INFO:tensorflow:step = 1, loss = 0.696266
INFO:tensorflow:probabilities = [[  9.99983907e-01   1.61303324e-05   1.69892422e-09]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  4.94141787e-01   1.54893698e-08   5.05858123e-01]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  9.99009490e-01   9.87224164e-04   3.16410842e-06]
 [  3.33333343e-01   3.33333343e-01   3.33333343e-01]
 [  9.67746794e-01   3.05207428e-02   1.73254113e-03]
 [  4.34234172e-01   3.18918586e-01   2.46847257e-01]
 [  9.99992132e-01   7.84881286e-06   4.54660087e-10]
 [  4.92559135e-01   1.44875889e-10   5.07440865e-0